In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,confusion_matrix,r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

In [2]:
def data_preproc(dummy_data):
    clean_data = dummy_data.copy(deep = True)
    clean_data.drop(columns=['Unnamed: 0','New_Price'], inplace=True)
    clean_data.dropna('index',inplace=True)
    clean_data = clean_data.reset_index(drop=True)
    clean_data.shape
    
    names = list(clean_data.Name)    
    for i in range(len(names)):
        names[i]=names[i].split(' ',1)[0]
        clean_data.Name = names
        clean_data.head()
        
    mileage = list(clean_data.Mileage)
    engine = list(clean_data.Engine)
    power = list(clean_data.Power)
    for i in range(len(names)):
        mileage[i] = mileage[i].split(' ',1)[0]
        engine[i] = engine[i].split(' ',1)[0]
        power[i] = power[i].split(' ',1)[0] 
    clean_data.Mileage = mileage
    clean_data.Engine = engine
    clean_data.Power = power
    
    clean_data["Kilometers_Driven"] = clean_data["Kilometers_Driven"].astype(float)
    clean_data["Mileage"] = clean_data["Mileage"].astype(float)
    clean_data["Engine"] = clean_data["Engine"].astype(float)
    
    clean_data = clean_data[clean_data.Seats != 0]
    
    idx = []
    lt = list(clean_data['Power'])
    for i in range(len(lt)):
        if(lt[i] == 'null'):
            idx.append(i)
    clean_data = clean_data.drop(idx)
    clean_data = clean_data.reset_index(drop=True)
    
    clean_data = clean_data[clean_data.Seats != 0]
    
    idx = []
    lt = list(clean_data['Power'])
    for i in range(len(lt)):
        if(lt[i] == 'null'):
            idx.append(i)
    clean_data = clean_data.drop(idx)
    clean_data = clean_data.reset_index(drop=True)

    clean_data['Power'] = clean_data['Power'].astype(float)
    
    clean_data['Year'] = pd.Categorical(clean_data['Year'])
    clean_data['Seats'] = pd.Categorical(clean_data['Seats'])
    clean_data = pd.get_dummies(clean_data, prefix_sep='_', drop_first=True) #문자를 숫자로 바꾸어 One-hot-encoding

    return clean_data

In [3]:
def makex(clean_data):
    x = clean_data.values 
    columns = clean_data.columns # 리스트값으로 열 레이블 반환
    #X값만
    scaler = preprocessing.MinMaxScaler() #최소 최대을 구함
    tmp = scaler.fit_transform(x) 
    clean_data = pd.DataFrame(tmp)
    clean_data.columns = columns
    x = clean_data.to_numpy()
    return x

In [4]:
dummy_train_data = pd.read_csv('./train-data.csv')
dummy_test_data = pd.read_csv('./test-data.csv')

In [5]:
clean_train_data = data_preproc(dummy_train_data)
clean_test_data = data_preproc(dummy_test_data)
print(clean_train_data.info())
print(clean_test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5844 entries, 0 to 5843
Data columns (total 79 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Kilometers_Driven          5844 non-null   float64
 1   Mileage                    5844 non-null   float64
 2   Engine                     5844 non-null   float64
 3   Power                      5844 non-null   float64
 4   Price                      5844 non-null   float64
 5   Name_Audi                  5844 non-null   uint8  
 6   Name_BMW                   5844 non-null   uint8  
 7   Name_Bentley               5844 non-null   uint8  
 8   Name_Chevrolet             5844 non-null   uint8  
 9   Name_Datsun                5844 non-null   uint8  
 10  Name_Fiat                  5844 non-null   uint8  
 11  Name_Force                 5844 non-null   uint8  
 12  Name_Ford                  5844 non-null   uint8  
 13  Name_Honda                 5844 non-null   uint8

In [6]:
#train 데이터 형성
y = clean_train_data[['Price']].to_numpy() #ndarray 형태로 변경
print(y)
clean_train_data = clean_train_data.drop(columns=['Price'])

[[ 1.75]
 [12.5 ]
 [ 4.5 ]
 ...
 [ 2.9 ]
 [ 2.65]
 [ 2.5 ]]


In [7]:
train_x = makex(clean_train_data)
test_x = makex(clean_test_data)

In [8]:
# 모델링 하기 = 다중선형회귀 사용
lr = LinearRegression(fit_intercept=True,normalize=True,copy_X=True)
lr.fit(train_x,y)

LinearRegression(normalize=True)

In [9]:
print(lr.score(train_x,y)) #성능치 표현 (0~1) 높아지면 좋은 것!

0.8069472191841415


In [10]:
y_predict = lr.predict(test_x)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 78 is different from 72)